In [1]:
import os
import re
import ast
import numpy as np
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

from trulens_eval import Tru
from trulens_eval import TruCustomApp
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.langchain import Langchain
from trulens_eval.tru_custom_app import instrument
from trulens_eval.feedback import prompts
import custom_prompts

from langchain_ibm import WatsonxLLM
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

from genai import Client, Credentials
from genai.extensions.langchain import LangChainInterface
from genai.schema import (
    DecodingMethod,
    TextGenerationParameters,
)


In [2]:
load_dotenv()

True

In [3]:
def watsonx_model(model_id="ibm-mistralai/mixtral-8x7b-instruct-v01-q", decoding_method='greedy', max_new_tokens=500, 
                  min_new_tokens=1, temperature=0.5, top_k=50, top_p=1, repetition_penalty=1):
    params = {
        GenParams.DECODING_METHOD: decoding_method,
        GenParams.MIN_NEW_TOKENS: min_new_tokens,
        GenParams.MAX_NEW_TOKENS: max_new_tokens,
        GenParams.RANDOM_SEED: 42,
        GenParams.TEMPERATURE: temperature,
        GenParams.TOP_K: top_k,
        GenParams.TOP_P: top_p,
        GenParams.REPETITION_PENALTY: repetition_penalty
    }
    ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None)
    project_id = os.getenv("PROJECT_ID", None)
    watsonx_llm = WatsonxLLM(
        model_id=model_id,
        url=ibm_cloud_url,
        project_id=project_id,
        params=params,
    )
    return watsonx_llm

In [4]:
def bam_model(model_id='mistralai/mixtral-8x7b-instruct-v0-1', decoding_method='greedy', max_new_tokens=1000, 
              min_new_tokens=1, temperature=0.5, top_k=50, top_p=1, repetition_penalty=1):

    if decoding_method == 'greedy':
        decoding_method = DecodingMethod.GREEDY
        parameters=TextGenerationParameters(
            decoding_method=decoding_method,
            max_new_tokens=max_new_tokens,
            min_new_tokens=min_new_tokens,
            repetition_penalty=repetition_penalty
        )
    else:
        decoding_method = DecodingMethod.SAMPLE
        parameters=TextGenerationParameters(
            decoding_method=decoding_method,
            max_new_tokens=max_new_tokens,
            min_new_tokens=min_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repetition_penalty=repetition_penalty
        )

    llm = LangChainInterface(
        model_id=model_id,
        client=Client(credentials=Credentials.from_env()),
        parameters=parameters,
    )

    return llm

In [5]:
# mixtral_llm = watsonx_model(model_id="ibm-mistralai/mixtral-8x7b-instruct-v01-q")
mixtral_llm = bam_model(model_id="mistralai/mixtral-8x7b-instruct-v0-1", repetition_penalty=1.1)

In [6]:
prompt = "Tell me about IBM."
result = mixtral_llm.invoke(prompt)
print(result)


IBM is a global technology and innovation company headquartered in Armonk, NY. It is the largest technology employer in the world, with more than 375,000 employees serving clients in 170 countries. IBM offers a wide range of technology and consulting services; a broad portfolio of middleware for collaboration, predictive analytics, software development and systems management; and the world's most advanced servers and supercomputers. Utilizing its business consulting, technology and R&D expertise, IBM helps clients become "smarter" as the planet becomes more digitally interconnected. IBM invests more than $6 billion a year in R&D, just completing its 20th consecutive year of patent leadership. IBM Research has received recognition beyond any commercial technology research organization and is home to 5 Nobel Laureates, 9 US National Medals of Technology, 5 US National Medals of Science, 6 Turing Awards, and 10 Inductees in US Inventors Hall of Fame. The company was founded in 1911, and 

In [7]:
df = pd.read_csv("data/ms-marco.csv")
df["contexts"] = [ast.literal_eval(ctx) for ctx in df["contexts"]]
df

,question,contexts
0,walgreens store sales average,[The average Walgreens salary ranges from appr...
1,how much do bartenders make,[A bartender’s income is comprised mostly of t...
2,what is a furuncle boil,"[Knowledge center. A boil, also known as a fur..."
3,what can urinalysis detect,[Urinalysis: One way to test for bladder cance...
4,what is vitamin a used for,[Since vitamin A is fat-soluble it is not need...
...,...,...
195,who wrote Nothing compares to you,[Nothing Compares 2 U is a song originally wr...
196,cost for relining dentures,[When dentures that used to fit are now loose ...
197,what is sherry wine made of,"[Sherry vinegar is made from sherry, a fortifi..."
198,dna in bacteria,[Bacterial DNA in Human Genomes. A new study f...


In [8]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# data = text_splitter.create_documents(df["contexts"].to_list(), metadatas=df[["question"]].to_dict(orient="records"))

def text_to_chunks(texts: str,
                   chunk_length: int = 100,
                   chunk_overlap: int = 25) -> list:
    """
    Splits the text into equally distributed chunks with 25-word overlap.
    Args:
        texts (str): Text to be converted into chunks.
        chunk_length (int): Maximum number of words in each chunk.
        chunk_overlap (int): Number of words to overlap between chunks.
    """
    words = texts.split(' ')
    n = len(words)
    chunks = []
    chunk_number = 1
    i = 0
    while i < n:  # Corrected the length check
        chunk = words[i: min(i + chunk_length, n)]
        i = i + chunk_length - chunk_overlap
        #print(len(chunk))
        chunk = ' '.join(chunk).strip()
        chunks.append({"text": chunk})
        chunk_number += 1
    return chunks

In [9]:
df['chunks'] = df['contexts'].apply(lambda x: [i['text'] for i in text_to_chunks(x[0])])
df['chunks'][0]

['The average Walgreens salary ranges from approximately $15,000 per year for Customer Service Associate / Cashier to $179,900 per year for District Manager. Average Walgreens hourly pay ranges from approximately $7.35 per hour for Laboratory Technician to $68.90 per hour for Pharmacy Manager. Salary information comes from 7,810 data points collected directly from employees, users, and jobs on Indeed.The average revenue in 2011 of a Starbuck Store was $1,078,000, up  from $1,011,000 in 2010.    The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.',
 'The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.    In 2008, the average ticket was flat (0.0% change).In fiscal 2014, Walgreens opened a total of 184 new locations and acquired 84 locations, for a net decrease of 273 after relocations and closings. How big are your stores? The average size for a typical Walgreens is about 14,5

In [10]:
from langchain.schema import Document

data = []
for i in range(len(df)):
    for j in df['chunks'][i]:
        doc = Document(
            metadata={
                "question": df['question'][i],
            },
            page_content=j)
        data.append(doc)

In [11]:
# Initialie the embedding model
model_name = "intfloat/e5-large-v2"
model_kwargs = {'device': 'cpu'}

embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs
)

In [12]:
vectorstore = FAISS.from_documents(data, embeddings_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}, verbose = False)

In [13]:
def prompt_generation(context, query):
    template = (
        "<s>"
        "[INST] \n"
        "Context: {context} \n"
        "- Take the context above and use that to answer questions in a detailed and professional way. \n"
        "- If you don't know the answer just say \"I don't know\".\n"
        "- Refrain from using any other knowledge other than the text provided.\n"
        "- Don't mention that you are answering from the text, impersonate as if this is coming from your knowledge\n"
        "- For the questions whose answer is not available in the provided context, just say \"I don't know\".\n"
        "Question: {query}? \n"
        "[/INST] \n"
        "</s>\n"
        "Answer: "
    )

    qa_template = PromptTemplate.from_template(template)
    return qa_template.format(context=context, query=query)

In [14]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [15]:
query= "how much do bartenders make"
docs = retriever.get_relevant_documents(query)

pretty_print_docs(docs)

Document 1:

hour between their wages and tips. According to bartending.org, bartenders in a high volume resort or establishment can earn $50,000 to $75,000 per year between hourly wages and tips. Indeed.com 2010 results show bartenders in restaurants at median salary rates can make a good salary per year: Bartender $73,000.
----------------------------------------------------------------------------------------------------
Document 2:

a bartender make in one year. If the service is good and the coversation is i … nteresting a bartender can make alot of money. Location is also a big factor.Best Answer: An average bartender makes about...2 to 3 dollars an hour. but all the money is made off tips depending on the popularity of the bar. I used to make $700 to $1000 a night. but that is in Atlanta. If the bar is busy and you are a good bartender you will make quite a bit. I dont know how much, because I live in a town with
------------------------------------------------------------------

In [16]:
context = "\n".join([doc.page_content for doc in docs])
prompt = prompt_generation(context, query)
print(prompt)

<s>[INST] 
Context: hour between their wages and tips. According to bartending.org, bartenders in a high volume resort or establishment can earn $50,000 to $75,000 per year between hourly wages and tips. Indeed.com 2010 results show bartenders in restaurants at median salary rates can make a good salary per year: Bartender $73,000.
a bartender make in one year. If the service is good and the coversation is i … nteresting a bartender can make alot of money. Location is also a big factor.Best Answer: An average bartender makes about...2 to 3 dollars an hour. but all the money is made off tips depending on the popularity of the bar. I used to make $700 to $1000 a night. but that is in Atlanta. If the bar is busy and you are a good bartender you will make quite a bit. I dont know how much, because I live in a town with
busy and you are a good bartender you will make quite a bit. I dont know how much, because I live in a town with a population of 2000 so there is not alot going on around he

In [17]:
result = mixtral_llm.invoke(prompt)
print(f"Answer: {result}")

Answer: 
Bartenders can make varying amounts of money based on several factors such as location, hourly wages, and tips earned. In high-volume resorts or establishments, bartenders can earn between $50,000 to $75,000 per year, including both hourly wages and tips. However, in restaurants, the median salary rate for bartenders can be around $73,000 per year. On average, bartenders make about $2 to $3 an hour in wages, but they mostly rely on tips, which can range from $20 to $300 or more per night. The amount of money a bartender can make ultimately depends on various factors, including the popularity of the bar, the quality of service, and the level of conversation they provide to customers.

Based on the information provided, it seems that bartenders in smaller towns may earn less than those working in larger cities or popular tourist destinations. For instance, in a town with a population of 2000, a bartender might earn a total of $100 to $200 per night (on a good night), depending o

In [18]:
class Processor:
    def __init__(self, retriever, llm):
        self._retriever = retriever
        self._llm = llm
    
    @instrument
    def retrieve_chunks(self, query, num_chunks):
        self._retriever.search_kwargs = {"k": num_chunks}
        docs = self._retriever.get_relevant_documents(query)
        docs = [doc.page_content for doc in docs]
        return docs
    
    @instrument
    def join_chunks(self, chunks):
        return "\n".join(chunks)
    
    @instrument
    def respond_to_query(self, query, num_chunks=3):
        chunks = self.retrieve_chunks(query, num_chunks=num_chunks)
        context = self.join_chunks(chunks)
        prompt = prompt_generation(context, query)
        retries_left = 3
        while True:
            try:
                answer = self._llm.invoke(prompt).strip()
                break
            except Exception as e:
                print("Error while generating answer", e)
                if retries_left>0:
                    retries_left -= 1
                    print("Retrying. Retries Remaining -", retries_left)
                else:
                    raise
        return answer
    

In [19]:
rag = Processor(retriever, mixtral_llm)

In [20]:
class IBMLangchain(Langchain):
    def _create_chat_completion(self, prompt = None, messages = None, **kwargs):
        if prompt is not None:
            # prompt += "\nANSWER:\n"
            prompt = f"[INST]\n{prompt}\n[/INST]"
            predict = self.endpoint.chain.invoke(prompt, **kwargs)
            predict = re.sub(r'Score: (\d+)/\d+', r'Score: \1', predict)

        elif messages is not None:
            prompt = messages[0]['content']
            # prompt += "\nANSWER:\n"
            prompt = f"[INST]\n{prompt}\n[/INST]"
            predict = self.endpoint.chain.invoke(prompt, **kwargs)
            predict = re.sub(r'Score: (\d+)/\d+', r'Score: \1', predict)

        else:
            raise ValueError("`prompt` or `messages` must be specified.")
        
        return predict
    
    def _groundedness_doc_in_out(self, premise: str, hypothesis: str) -> str:
        """
        An LLM prompt using the entire document for premise and entire statement
        document for hypothesis.

        Args:
            premise (str): A source document
            hypothesis (str): A statement to check

        Returns:
            str: An LLM response using a scorecard template
        """
        assert self.endpoint is not None, "Endpoint is not set."

        return self.endpoint.run_in_pace(
            func=self._create_chat_completion,
            prompt=str.format(custom_prompts.LLM_GROUNDEDNESS_FULL_SYSTEM,) +
            str.format(
                prompts.LLM_GROUNDEDNESS_FULL_PROMPT,
                premise=premise,
                hypothesis=hypothesis
            )
        )

In [21]:
eval_llm = bam_model(model_id="mistralai/mixtral-8x7b-instruct-v0-1", min_new_tokens=1, max_new_tokens=1000, repetition_penalty=1.1)
# eval_llm = bam_model(model_id="meta-llama/llama-2-70b-chat", max_new_tokens=1000, repetition_penalty=1.1)

langchain_provider = IBMLangchain(chain = eval_llm)

In [22]:
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [23]:
# Question/statement relevance between question and each context chunk.
f_qs_relevance = (
    Feedback(
        langchain_provider.qs_relevance_with_cot_reasons,
        name="Context Relevance"
    )
    .on_input()
    .on(Select.RecordCalls.retrieve_chunks.rets[:])
    .aggregate(np.mean)
)

# Define a groundedness feedback function
grounded = Groundedness(groundedness_provider=langchain_provider)
f_groundedness = (
    Feedback(
        grounded.groundedness_measure_with_cot_reasons,
        name="Groundedness"
    )
    .on(Select.RecordCalls.join_chunks.rets) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(
    langchain_provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input().on_output()

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.retrieve_chunks.rets[:] .
✅ In Groundedness, input source will be set to __record__.app.join_chunks.rets .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [24]:
tru_rag = TruCustomApp(rag,
    app_id = 'RAG Pipeline',
    feedbacks = [f_qs_relevance, f_groundedness, f_qa_relevance])

In [25]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://9.199.156.133:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
with tru_rag as recording:
    for query in tqdm(df["question"], total=len(df)):
        ans = rag.respond_to_query(query)

In [27]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
RAG Pipeline,0.830626,0.741158,0.986432,6.895,0.0
